# はじめに / Credit
このNotebookの実装コードは、Jay Mody氏による「NumPyのみで書かれたGPT-2実装」である **[picoGPT](https://github.com/jaymody/picoGPT)** をベースにしています。

本シリーズの教育的な目的のために、オリジナルのPythonスクリプトをJupyter Notebook形式に再構成（リファクタリング）し、日本語による詳細なコード解説と、中間変数の形状(Shape)を確認するためのステップを追加しました。

* **Original Repository**: [jaymody/picoGPT](https://github.com/jaymody/picoGPT)
* **Original Author**: Jay Mody
* **License**: MIT License
* **Copyright**: (c) 2023 Jay Mody

---

<a href="https://colab.research.google.com/github/nncliff/qwen-32B/blob/main/chapter-0/picoGPT_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PicoGPT 推論ノートブック

このノートブックでは、picoGPT実装を使用してGPT-2をロードし、推論を実行する方法を説明します。

picoGPTは純粋なNumPyで実装されたGPT-2の最小実装であり、Transformerベースの言語モデルのコア概念を理解しやすくしています。

**特徴:**
- 事前学習済みGPT-2の重みをロード
- BPEトークナイゼーション
- 貪欲法（グリーディデコーディング）によるテキスト生成

## 1. セットアップとインポート

In [1]:
import os
import json
import re
from functools import lru_cache

import numpy as np
import regex
import requests
import tensorflow as tf
from tqdm import tqdm

## 2. BPEトークナイザー（エンコーダー）

GPT-2トークナイザーはByte Pair Encoding（BPE）を使用してテキストをトークンに変換します。

In [2]:
@lru_cache()
def bytes_to_unicode():
    """
    Returns list of utf-8 byte and a corresponding list of unicode strings.
    The reversible bpe codes work on unicode strings.
    """
    bs = list(range(ord("!"), ord("~") + 1)) + list(range(ord("¡"), ord("¬") + 1)) + list(range(ord("®"), ord("ÿ") + 1))
    cs = bs[:]
    n = 0
    for b in range(2**8):
        if b not in bs:
            bs.append(b)
            cs.append(2**8 + n)
            n += 1
    cs = [chr(n) for n in cs]
    return dict(zip(bs, cs))


def get_pairs(word):
    """Return set of symbol pairs in a word."""
    pairs = set()
    prev_char = word[0]
    for char in word[1:]:
        pairs.add((prev_char, char))
        prev_char = char
    return pairs


class Encoder:
    """BPE Encoder/Decoder for GPT-2."""

    def __init__(self, encoder, bpe_merges, errors="replace"):
        self.encoder = encoder
        self.decoder = {v: k for k, v in self.encoder.items()}
        self.errors = errors
        self.byte_encoder = bytes_to_unicode()
        self.byte_decoder = {v: k for k, v in self.byte_encoder.items()}
        self.bpe_ranks = dict(zip(bpe_merges, range(len(bpe_merges))))
        self.cache = {}
        self.pat = regex.compile(r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+""")

    def bpe(self, token):
        if token in self.cache:
            return self.cache[token]
        word = tuple(token)
        pairs = get_pairs(word)

        if not pairs:
            return token

        while True:
            bigram = min(pairs, key=lambda pair: self.bpe_ranks.get(pair, float("inf")))
            if bigram not in self.bpe_ranks:
                break
            first, second = bigram
            new_word = []
            i = 0
            while i < len(word):
                try:
                    j = word.index(first, i)
                    new_word.extend(word[i:j])
                    i = j
                except:
                    new_word.extend(word[i:])
                    break

                if word[i] == first and i < len(word) - 1 and word[i + 1] == second:
                    new_word.append(first + second)
                    i += 2
                else:
                    new_word.append(word[i])
                    i += 1
            new_word = tuple(new_word)
            word = new_word
            if len(word) == 1:
                break
            else:
                pairs = get_pairs(word)
        word = " ".join(word)
        self.cache[token] = word
        return word

    def encode(self, text):
        """Encode text to token IDs."""
        bpe_tokens = []
        for token in regex.findall(self.pat, text):
            token = "".join(self.byte_encoder[b] for b in token.encode("utf-8"))
            bpe_tokens.extend(self.encoder[bpe_token] for bpe_token in self.bpe(token).split(" "))
        return bpe_tokens

    def decode(self, tokens):
        """Decode token IDs back to text."""
        text = "".join([self.decoder[token] for token in tokens])
        text = bytearray([self.byte_decoder[c] for c in text]).decode("utf-8", errors=self.errors)
        return text


def get_encoder(model_name, models_dir):
    """Load the BPE encoder from files."""
    with open(os.path.join(models_dir, model_name, "encoder.json"), "r") as f:
        encoder = json.load(f)
    with open(os.path.join(models_dir, model_name, "vocab.bpe"), "r", encoding="utf-8") as f:
        bpe_data = f.read()
    bpe_merges = [tuple(merge_str.split()) for merge_str in bpe_data.split("\n")[1:-1]]
    return Encoder(encoder=encoder, bpe_merges=bpe_merges)

## 3. モデルロードユーティリティ

OpenAIからGPT-2の重みをダウンロードし、TensorFlowチェックポイントからロードする関数群です。

In [3]:
def download_gpt2_files(model_size, model_dir):
    """Download GPT-2 model files from OpenAI."""
    assert model_size in ["124M", "355M", "774M", "1558M"]
    for filename in [
        "checkpoint",
        "encoder.json",
        "hparams.json",
        "model.ckpt.data-00000-of-00001",
        "model.ckpt.index",
        "model.ckpt.meta",
        "vocab.bpe",
    ]:
        url = "https://openaipublic.blob.core.windows.net/gpt-2/models"
        r = requests.get(f"{url}/{model_size}/{filename}", stream=True)
        r.raise_for_status()

        with open(os.path.join(model_dir, filename), "wb") as f:
            file_size = int(r.headers["content-length"])
            chunk_size = 1000
            with tqdm(
                ncols=100,
                desc="Fetching " + filename,
                total=file_size,
                unit_scale=True,
                unit="b",
            ) as pbar:
                for chunk in r.iter_content(chunk_size=chunk_size):
                    f.write(chunk)
                    pbar.update(chunk_size)


def load_gpt2_params_from_tf_ckpt(tf_ckpt_path, hparams):
    """Load GPT-2 parameters from TensorFlow checkpoint."""
    def set_in_nested_dict(d, keys, val):
        if not keys:
            return val
        if keys[0] not in d:
            d[keys[0]] = {}
        d[keys[0]] = set_in_nested_dict(d[keys[0]], keys[1:], val)
        return d

    params = {"blocks": [{} for _ in range(hparams["n_layer"])]}
    for name, _ in tf.train.list_variables(tf_ckpt_path):
        array = np.squeeze(tf.train.load_variable(tf_ckpt_path, name))
        name = name[len("model/") :]
        if name.startswith("h"):
            m = re.match(r"h([0-9]+)/(.*)", name)
            n = int(m[1])
            sub_name = m[2]
            set_in_nested_dict(params["blocks"][n], sub_name.split("/"), array)
        else:
            set_in_nested_dict(params, name.split("/"), array)

    return params


def load_encoder_hparams_and_params(model_size, models_dir):
    """Load encoder, hyperparameters, and model parameters."""
    assert model_size in ["124M", "355M", "774M", "1558M"]

    model_dir = os.path.join(models_dir, model_size)
    tf_ckpt_path = tf.train.latest_checkpoint(model_dir)
    if not tf_ckpt_path:  # download files if necessary
        os.makedirs(model_dir, exist_ok=True)
        download_gpt2_files(model_size, model_dir)
        tf_ckpt_path = tf.train.latest_checkpoint(model_dir)

    encoder = get_encoder(model_size, models_dir)
    hparams = json.load(open(os.path.join(model_dir, "hparams.json")))
    params = load_gpt2_params_from_tf_ckpt(tf_ckpt_path, hparams)

    return encoder, hparams, params

## 4. GPT-2モデルコンポーネント

これらは純粋なNumPyで実装されたGPT-2アーキテクチャのコアビルディングブロックです。

### GELU（ガウス誤差線形ユニット）

GELUはGPT-2（およびBERT、GPT-3などの多くの最新Transformer）で使用される活性化関数です。0で急激なカットオフを持つReLUとは異なり、GELUは滑らかな確率的ゲーティングメカニズムを提供します。

**正確な定義:**

$$\text{GELU}(x) = x \cdot \Phi(x) = x \cdot P(X \leq x)$$

ここで$\Phi(x)$は標準正規分布の累積分布関数（CDF）です。

**近似（実際に使用される）:**

正確なCDFの計算はコストが高いため、GPT-2では$\tanh$に基づく高速近似を使用します：

$$\text{GELU}(x) \approx 0.5 \cdot x \cdot \left(1 + \tanh\left(\sqrt{\frac{2}{\pi}} \cdot (x + 0.044715 \cdot x^3)\right)\right)$$

**なぜReLUではなくGELUか？**
- **滑らか**: すべての点で微分可能（0でキンクなし）
- **非単調**: 小さな負の値が小さな正の出力を持つことがある
- **確率的解釈**: 入力値に依存する確率でベルヌーイマスクを乗算すると見なすことができる

### ソフトマックス

ソフトマックスは実数のベクトル（ロジット）を確率分布に変換します。アテンションメカニズムと最終出力層で使用されます。

**定義:**

ベクトル$\mathbf{x} = [x_1, x_2, ..., x_n]$に対して：

$$\text{softmax}(x_i) = \frac{e^{x_i}}{\sum_{j=1}^{n} e^{x_j}}$$

**性質:**
- 出力値は$(0, 1)$の範囲
- すべての出力の合計は1: $\sum_i \text{softmax}(x_i) = 1$
- 順序を保存: $x_i > x_j$ならば$\text{softmax}(x_i) > \text{softmax}(x_j)$

**数値安定性の問題:**

$e^{x_i}$を直接計算すると、大きな$x$でオーバーフローが発生する可能性があります。解決策は最大値を引くことです：

$$\text{softmax}(x_i) = \frac{e^{x_i - \max(\mathbf{x})}}{\sum_{j=1}^{n} e^{x_j - \max(\mathbf{x})}}$$

これは数学的に同等（$e^{-\max(\mathbf{x})}$が打ち消し合う）ですが、最大の指数が$e^0 = 1$になるためオーバーフローを防ぎます。

### レイヤー正規化

レイヤー正規化は特徴次元全体で活性化を正規化することで訓練を安定させます。バッチ正規化（バッチ全体で正規化）とは異なり、レイヤー正規化は個々のサンプルに対して機能します。

**定義:**

次元$d$の入力ベクトル$\mathbf{x} = [x_1, x_2, ..., x_d]$に対して：

$$\text{LayerNorm}(\mathbf{x}) = \gamma \cdot \frac{\mathbf{x} - \mu}{\sqrt{\sigma^2 + \epsilon}} + \beta$$

ここで：
- $\mu = \frac{1}{d}\sum_{i=1}^{d} x_i$ は平均
- $\sigma^2 = \frac{1}{d}\sum_{i=1}^{d} (x_i - \mu)^2$ は分散
- $\gamma$（スケール）と$\beta$（シフト）は学習可能なパラメータ
- $\epsilon$は数値安定性のための小さな定数（例：$10^{-5}$）

**なぜレイヤー正規化か？**
- **勾配の安定化**: 活性化が大きくなりすぎたり小さくなりすぎたりするのを防ぐ
- **より深いネットワークを可能に**: 多くの層を持つTransformerの訓練に不可欠
- **バッチサイズに依存しない**: 訓練と推論で同じように動作

**GPT-2での使用:**
- アテンションとFFNの**前**に適用（Pre-LNアーキテクチャ）
- 各Transformerブロックには2つのレイヤー正規化がある：`ln_1`（アテンション前）と`ln_2`（FFN前）

### 線形層（全結合層）

線形変換はニューラルネットワークの基本的なビルディングブロックです。全結合層、密層、またはアフィン変換とも呼ばれます。

**定義:**

入力$\mathbf{x} \in \mathbb{R}^{d_{in}}$、重み行列$\mathbf{W} \in \mathbb{R}^{d_{in} \times d_{out}}$、バイアス$\mathbf{b} \in \mathbb{R}^{d_{out}}$に対して：

$$\text{Linear}(\mathbf{x}) = \mathbf{x} \mathbf{W} + \mathbf{b}$$

**形状変換:**
- 入力: $[m, d_{in}]$（$m$個のベクトルのバッチ）
- 重み: $[d_{in}, d_{out}]$
- バイアス: $[d_{out}]$
- 出力: $[m, d_{out}]$

**GPT-2での線形層の用途:**
- **QKV射影**（`c_attn`）: 入力をクエリ、キー、バリューベクトルに射影
- **出力射影**（`c_proj`）: アテンション出力を埋め込み次元に射影
- **FFN上方射影**（`c_fc`）: $d_{model}$から$4 \cdot d_{model}$に拡張
- **FFN下方射影**（`c_proj`）: $d_{model}$に圧縮

**注:** NumPy/Pythonの`@`演算子は行列乗算を実行します。

In [4]:
def gelu(x):
    """Gaussian Error Linear Unit activation function."""
    return 0.5 * x * (1 + np.tanh(np.sqrt(2 / np.pi) * (x + 0.044715 * x**3)))


def softmax(x):
    """Numerically stable softmax function."""
    exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=-1, keepdims=True)


def layer_norm(x, g, b, eps: float = 1e-5):
    """Layer normalization."""
    mean = np.mean(x, axis=-1, keepdims=True)
    variance = np.var(x, axis=-1, keepdims=True)
    x = (x - mean) / np.sqrt(variance + eps)  # normalize x to have mean=0 and var=1 over last axis
    return g * x + b  # scale and offset with gamma/beta params


def linear(x, w, b):  # [m, in], [in, out], [out] -> [m, out]
    """Linear transformation."""
    return x @ w + b

### フィードフォワードネットワーク（FFN / MLP）

フィードフォワードネットワーク（MLPまたは位置ごとのフィードフォワードとも呼ばれる）は、シーケンス内の各位置に独立して適用されます。GELUの活性化を挟んだ2つの線形変換で構成されます。

**定義:**

$$\text{FFN}(\mathbf{x}) = \text{Linear}_2(\text{GELU}(\text{Linear}_1(\mathbf{x})))$$

より明示的に：

$$\text{FFN}(\mathbf{x}) = \text{GELU}(\mathbf{x} \mathbf{W}_1 + \mathbf{b}_1) \mathbf{W}_2 + \mathbf{b}_2$$

**アーキテクチャ:**
```
入力 [n_seq, d_model]
    ↓
線形（上方射影）: d_model → 4 × d_model
    ↓
GELU活性化
    ↓
線形（下方射影）: 4 × d_model → d_model
    ↓
出力 [n_seq, d_model]
```

**なぜ4倍拡張か？**
- 中間次元は通常$4 \times d_{model}$（例：GPT-2 smallでは768 → 3072）
- この「ボトルネック」設計により、ネットワークはより豊かな表現を学習できる
- 拡張により非線形変換のためのより多くの容量を提供

**GPT-2での使用:**
- `c_fc`: 上方射影の重み（次元を拡張）
- `c_proj`: 下方射影の重み（圧縮）
- 各Transformerブロックでレイヤー正規化の後に適用

### スケールドドットプロダクトアテンション

アテンションは、各出力を生成する際に入力の関連部分に焦点を当てることを可能にするコアメカニズムです。クエリとキー間の類似度によって重みが決定される値の重み付き和を計算します。

**定義:**

$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}} + \text{mask}\right) V$$

ここで：
- $Q \in \mathbb{R}^{n_q \times d_k}$ — クエリ行列（探しているもの）
- $K \in \mathbb{R}^{n_k \times d_k}$ — キー行列（マッチングする対象）
- $V \in \mathbb{R}^{n_k \times d_v}$ — バリュー行列（取得するもの）
- $d_k$ — キーの次元（スケーリングに使用）
- mask — 将来のトークンへのアテンションを防ぐ因果マスク

**ステップバイステップの分解:**

1. **アテンションスコアの計算**: $QK^T$は各クエリとキー間の類似度を与える
2. **スケーリング**: 大きな$d_k$でのソフトマックス飽和を防ぐため$\sqrt{d_k}$で割る
3. **マスク**: アテンションすべきでない位置に$-\infty$（または$-10^{10}$）を加える
4. **ソフトマックス**: スコアを確率に変換（行の合計は1）
5. **重み付き和**: $V$を掛けて出力を得る

**なぜ$\sqrt{d_k}$でスケーリングするのか？**

スケーリングなしでは、$d_k$が大きい場合、内積$q \cdot k$の大きさが増大し、ソフトマックスを非常に小さな勾配を持つ領域に押し込みます。$\sqrt{d_k}$でスケーリングすることで分散を安定に保ちます。

**因果マスク（GPT用）:**

自己回帰モデルでは、位置$i$が位置$\leq i$にのみアテンションできるように下三角マスクを使用します：

$$\text{mask}_{ij} = \begin{cases} 0 & \text{if } j \leq i \\ -\infty & \text{if } j > i \end{cases}$$

### マルチヘッドアテンション（MHA）

マルチヘッドアテンションは、それぞれ異なる学習済み射影を持つ複数のアテンション操作を並列に実行します。これにより、モデルは異なる表現サブスペースからの情報に同時にアテンションを向けることができます。

**定義:**

$$\text{MHA}(X) = \text{Concat}(\text{head}_1, ..., \text{head}_h) W^O$$

各ヘッドは：

$$\text{head}_i = \text{Attention}(XW_i^Q, XW_i^K, XW_i^V)$$

**アーキテクチャ:**
```
入力 X [n_seq, d_model]
    ↓
線形射影 → Q, K, V [n_seq, 3 × d_model]
    ↓
n_headヘッドに分割 → 各ヘッドの次元 d_k = d_model / n_head
    ↓
各ヘッドで並列アテンション（因果マスク付き）
    ↓
ヘッドを連結 → [n_seq, d_model]
    ↓
出力射影 → [n_seq, d_model]
```

**なぜ複数のヘッドか？**

- **異なるアテンションパターン**: 各ヘッドは異なる側面（例：構文vs意味論、近くvs遠くのトークン）に焦点を当てることを学習できる
- **より豊かな表現**: 同じ総パラメータ数の単一アテンションよりも表現力が高い
- **並列計算**: すべてのヘッドが独立して計算され、効率的な並列化が可能

**GPT-2の設定:**
| モデル | $d_{model}$ | $n_{head}$ | $d_k = d_{model}/n_{head}$ |
|-------|-------------|------------|----------------------------|
| 124M  | 768         | 12         | 64                         |
| 355M  | 1024        | 16         | 64                         |
| 774M  | 1280        | 20         | 64                         |
| 1558M | 1600        | 25         | 64                         |

**コード内での表現:**
- `c_attn`: 結合QKV射影の重み $[d_{model}, 3 \times d_{model}]$
- `c_proj`: 出力射影の重み $[d_{model}, d_{model}]$
- `np.split(..., 3)`: 射影をQ, K, Vに分割
- `np.split(..., n_head)`: 各々を複数のヘッドに分割

In [5]:
def ffn(x, c_fc, c_proj):  # [n_seq, n_embd] -> [n_seq, n_embd]
    """Feed-forward network (MLP) in transformer."""
    # project up
    a = gelu(linear(x, **c_fc))  # [n_seq, n_embd] -> [n_seq, 4*n_embd]

    # project back down
    x = linear(a, **c_proj)  # [n_seq, 4*n_embd] -> [n_seq, n_embd]

    return x


def attention(q, k, v, mask):  # [n_q, d_k], [n_k, d_k], [n_k, d_v], [n_q, n_k] -> [n_q, d_v]
    """Scaled dot-product attention."""
    return softmax(q @ k.T / np.sqrt(q.shape[-1]) + mask) @ v


def mha(x, c_attn, c_proj, n_head):  # [n_seq, n_embd] -> [n_seq, n_embd]
    """Multi-head attention."""
    # qkv projection
    x = linear(x, **c_attn)  # [n_seq, n_embd] -> [n_seq, 3*n_embd]

    # split into qkv
    qkv = np.split(x, 3, axis=-1)  # [n_seq, 3*n_embd] -> [3, n_seq, n_embd]

    # split into heads
    qkv_heads = list(map(lambda x: np.split(x, n_head, axis=-1), qkv))  # [3, n_seq, n_embd] -> [3, n_head, n_seq, n_embd/n_head]

    # causal mask to hide future inputs from being attended to
    causal_mask = (1 - np.tri(x.shape[0], dtype=x.dtype)) * -1e10  # [n_seq, n_seq]

    # perform attention over each head
    out_heads = [attention(q, k, v, causal_mask) for q, k, v in zip(*qkv_heads)]  # [3, n_head, n_seq, n_embd/n_head] -> [n_head, n_seq, n_embd/n_head]

    # merge heads
    x = np.hstack(out_heads)  # [n_head, n_seq, n_embd/n_head] -> [n_seq, n_embd]

    # out projection
    x = linear(x, **c_proj)  # [n_seq, n_embd] -> [n_seq, n_embd]

    return x

### Transformerブロック

Transformerブロックは、GPT-2の基本的な繰り返しユニットです。各ブロックは、自己アテンション（位置間で情報を混合）とフィードフォワードネットワーク（各位置を独立して処理）を組み合わせ、残差接続とレイヤー正規化で接続されています。

**定義（Pre-LNアーキテクチャ）:**

GPT-2は**Pre-LayerNorm**バリアントを使用し、正規化は各サブレイヤーの*前*に適用されます：

$$\mathbf{x} = \mathbf{x} + \text{MHA}(\text{LayerNorm}(\mathbf{x}))$$
$$\mathbf{x} = \mathbf{x} + \text{FFN}(\text{LayerNorm}(\mathbf{x}))$$

**アーキテクチャ図:**
```
入力 x [n_seq, d_model]
    │
    ├───────────────────────────┐
    ↓                           │ (残差)
LayerNorm (ln_1)                │
    ↓                           │
マルチヘッドアテンション         │
    ↓                           │
    + ←─────────────────────────┘
    │
    ├───────────────────────────┐
    ↓                           │ (残差)
LayerNorm (ln_2)                │
    ↓                           │
フィードフォワードネットワーク   │
    ↓                           │
    + ←─────────────────────────┘
    ↓
出力 [n_seq, d_model]
```

**主要コンポーネント:**

| コンポーネント | 目的 | パラメータ |
|-----------|---------|------------|
| `ln_1` | アテンション前の正規化 | $\gamma_1, \beta_1$ |
| `attn` | マルチヘッド自己アテンション | $W^Q, W^K, W^V, W^O$ |
| `ln_2` | FFN前の正規化 | $\gamma_2, \beta_2$ |
| `mlp` | 位置ごとのフィードフォワード | $W_1, b_1, W_2, b_2$ |

**なぜ残差接続か？**

`+`操作は、ResNetで導入された**残差（スキップ）接続**です：

$$\text{output} = \text{input} + F(\text{input})$$

利点：
- **勾配の流れ**: 勾配がスキップ接続を通じて直接流れ、深いネットワークでの勾配消失を防ぐ
- **恒等写像**: $F$がゼロを出力する場合、レイヤーは恒等関数になり、最適化が容易
- **インクリメンタル学習**: 各サブレイヤーは表現に「改良」を加えることを学習

**Pre-LN vs Post-LN:**

| 側面 | Pre-LN (GPT-2) | Post-LN (元のTransformer) |
|--------|----------------|-------------------------------|
| 式 | $x + \text{SubLayer}(\text{LN}(x))$ | $\text{LN}(x + \text{SubLayer}(x))$ |
| 訓練 | より安定 | ウォームアップなしでは不安定になりうる |
| 出力スケール | 深さとともに増加 | 各層で正規化 |

**GPT-2のスタック:**
- 124M: 12ブロック
- 355M: 24ブロック
- 774M: 36ブロック
- 1558M: 48ブロック

### GPT-2フォワードパス

`gpt2()`関数はモデルの完全なフォワードパスです。トークンIDを入力として受け取り、各位置での次のトークンに対する語彙全体のロジット（正規化されていない確率）を出力します。

**定義:**

$$\text{GPT-2}(\mathbf{x}) = \text{LayerNorm}(\text{Blocks}(\mathbf{E}_{token} + \mathbf{E}_{pos})) \cdot \mathbf{W}_E^T$$

**ステップバイステップ:**

1. **トークン埋め込み**: 各入力トークンを埋め込み行列で検索
   $$\mathbf{h}_0^{(token)} = \mathbf{W}_E[\text{input\_ids}] \in \mathbb{R}^{n_{seq} \times d_{model}}$$

2. **位置埋め込み**: 位置情報を追加
   $$\mathbf{h}_0 = \mathbf{h}_0^{(token)} + \mathbf{W}_P[0, 1, ..., n_{seq}-1]$$

3. **Transformerブロック**: $L$個のTransformerブロックを通過
   $$\mathbf{h}_l = \text{TransformerBlock}_l(\mathbf{h}_{l-1}) \quad \text{for } l = 1, ..., L$$

4. **最終レイヤー正規化**: 出力を正規化
   $$\mathbf{h}_{out} = \text{LayerNorm}(\mathbf{h}_L)$$

5. **語彙への射影**: **共有**埋め込み行列を使用してロジットを計算
   $$\text{logits} = \mathbf{h}_{out} \cdot \mathbf{W}_E^T \in \mathbb{R}^{n_{seq} \times n_{vocab}}$$

**アーキテクチャ概要:**
```
入力トークンID [n_seq]
    ↓
トークン埋め込み (wte): ルックアップテーブル [n_vocab, d_model]
    +
位置埋め込み (wpe): ルックアップテーブル [n_ctx, d_model]
    ↓
隠れ状態 [n_seq, d_model]
    ↓
┌─────────────────────────────────┐
│   Transformerブロック 0          │
│   (ln_1 → MHA → ln_2 → FFN)     │
└─────────────────────────────────┘
    ↓
    ... (n_layer回繰り返し)
    ↓
┌─────────────────────────────────┐
│   Transformerブロック L-1        │
└─────────────────────────────────┘
    ↓
最終LayerNorm (ln_f)
    ↓
wte.Tとの行列積（重み共有）
    ↓
ロジット [n_seq, n_vocab]
```

**主要概念:**

| 概念 | 説明 |
|---------|-------------|
| **トークン埋め込み** (`wte`) | 各トークンIDを密ベクトルにマッピングする学習済みルックアップテーブル |
| **位置埋め込み** (`wpe`) | 位置情報（0からn_ctx-1）をエンコードする学習済みベクトル |
| **重み共有** | 出力射影は別の重みの代わりに`wte.T`を再利用し、パラメータを削減 |
| **最終LayerNorm** (`ln_f`) | すべてのブロックの後、語彙への射影前に適用 |

**重み共有の洞察:**

GPT-2は**重み共有**を使用 — 同じ埋め込み行列$\mathbf{W}_E$が以下に使用されます：
- **入力**: トークンID → ベクトル（行検索）
- **出力**: ベクトル → ロジット（転置との行列積）

これによりパラメータが削減され、類似トークンが類似のロジットを持つ意味のある出力空間が作成されます。

**GPT-2モデルサイズ:**

| モデル | $n_{layer}$ | $d_{model}$ | $n_{head}$ | $n_{ctx}$ | $n_{vocab}$ |
|-------|-------------|-------------|------------|-----------|-------------|
| 124M  | 12          | 768         | 12         | 1024      | 50257       |
| 355M  | 24          | 1024        | 16         | 1024      | 50257       |
| 774M  | 36          | 1280        | 20         | 1024      | 50257       |
| 1558M | 48          | 1600        | 25         | 1024      | 50257       |

In [6]:
def transformer_block(x, mlp, attn, ln_1, ln_2, n_head):  # [n_seq, n_embd] -> [n_seq, n_embd]
    """A single transformer block."""
    # multi-head causal self attention
    x = x + mha(layer_norm(x, **ln_1), **attn, n_head=n_head)  # [n_seq, n_embd] -> [n_seq, n_embd]

    # position-wise feed forward network
    x = x + ffn(layer_norm(x, **ln_2), **mlp)  # [n_seq, n_embd] -> [n_seq, n_embd]

    return x


def gpt2(inputs, wte, wpe, blocks, ln_f, n_head):  # [n_seq] -> [n_seq, n_vocab]
    """GPT-2 forward pass."""
    # token + positional embeddings
    x = wte[inputs] + wpe[range(len(inputs))]  # [n_seq] -> [n_seq, n_embd]

    # forward pass through n_layer transformer blocks
    for block in blocks:
        x = transformer_block(x, **block, n_head=n_head)  # [n_seq, n_embd] -> [n_seq, n_embd]

    # projection to vocab
    x = layer_norm(x, **ln_f)  # [n_seq, n_embd] -> [n_seq, n_embd]
    return x @ wte.T  # [n_seq, n_embd] -> [n_seq, n_vocab]

## 5. テキスト生成関数

### 自己回帰生成

`generate()`関数は**自己回帰テキスト生成**を実装します — モデルは一度に1つのトークンを予測し、その予測を入力としてフィードバックして次のトークンを予測します。

**定義:**

入力トークン$\mathbf{x} = [x_1, x_2, ..., x_n]$が与えられた場合、$T$個の新しいトークンを生成：

$$x_{n+t} = \arg\max_{v \in \mathcal{V}} P(v \mid x_1, ..., x_{n+t-1}) \quad \text{for } t = 1, ..., T$$

ここで$\mathcal{V}$は語彙であり、確率はロジットに対するソフトマックスから来ます。

**ステップバイステップアルゴリズム:**

```
入力: プロンプトトークン [x₁, x₂, ..., xₙ], トークン数 T
出力: 生成されたトークン [xₙ₊₁, xₙ₊₂, ..., xₙ₊ₜ]

for t = 1 to T:
    1. フォワードパス: logits = GPT2([x₁, ..., xₙ₊ₜ₋₁])
    2. 最後の位置を取得: last_logits = logits[-1]  # [n_vocab]
    3. 次のトークンを選択: xₙ₊ₜ = argmax(last_logits)
    4. シーケンスに追加: [x₁, ..., xₙ₊ₜ₋₁] → [x₁, ..., xₙ₊ₜ]

return [xₙ₊₁, ..., xₙ₊ₜ]
```

**視覚的表現:**

```
ステップ1: "The cat" → GPT-2 → logits → argmax → "sat"
ステップ2: "The cat sat" → GPT-2 → logits → argmax → "on"
ステップ3: "The cat sat on" → GPT-2 → logits → argmax → "the"
...
```

**なぜ`logits[-1]`のみを使用するのか？**

GPT-2はシーケンス内の**すべての位置**に対してロジットを出力します$[n_{seq}, n_{vocab}]$。しかし生成では、最後のトークンの**後**に来るものを予測することだけに関心があるため、`logits[-1]`を取ります。

**デコーディング戦略:**

| 戦略 | 式 | 特性 |
|----------|---------|------------|
| **貪欲法**（ここで使用） | $x = \arg\max(logits)$ | 決定論的、高速、繰り返しになりやすい |
| **温度サンプリング** | $x \sim \text{softmax}(logits / \tau)$ | $\tau < 1$: より鋭く、$\tau > 1$: よりランダム |
| **Top-kサンプリング** | 上位$k$トークンからサンプル | 最も可能性の高い選択肢に制限 |
| **Top-p（核）** | 累積$p$を持つ最小セットからサンプル | 適応的な語彙サイズ |

**貪欲デコーディング:**

この実装は**貪欲デコーディング**を使用 — 常に最も確率の高い次のトークンを選択：

$$x_{next} = \arg\max_{v} \text{logits}[v]$$

利点：
- シンプルで高速
- 決定論的（同じ入力 → 同じ出力）

欠点：
- 繰り返しループに陥りやすい
- より良いシーケンスを見逃す可能性（探索なし）
- グローバル最適ではない（局所的な決定が最適でないテキストにつながる可能性）

**計算上の注意:**

この素朴な実装は各ステップでシーケンス全体を再計算します。実際には、**KVキャッシング**が中間アテンション状態を保存して冗長な計算を避け、トークンあたり$O(n^2)$ではなく$O(n)$で生成を行います。

In [7]:
def generate(inputs, params, n_head, n_tokens_to_generate):
    """Generate tokens autoregressively using greedy decoding."""
    for _ in tqdm(range(n_tokens_to_generate), desc="Generating"):
        logits = gpt2(inputs, **params, n_head=n_head)  # model forward pass
        next_id = np.argmax(logits[-1])  # greedy sampling
        inputs.append(int(next_id))  # append prediction to input

    return inputs[len(inputs) - n_tokens_to_generate:]  # only return generated ids

## 6. モデルの重みとトークナイザーのロード

事前学習済みGPT-2モデルをロードします。モデルが存在しない場合は自動的にダウンロードされます。

利用可能なモデルサイズ：
- `124M` - Small（デフォルト）
- `355M` - Medium
- `774M` - Large
- `1558M` - XL

In [8]:
# Configuration
MODEL_SIZE = "124M"  # Choose from: "124M", "355M", "774M", "1558M"
MODELS_DIR = "models"  # Directory to store downloaded models

# Load encoder (tokenizer), hyperparameters, and model parameters
print(f"Loading GPT-2 {MODEL_SIZE} model...")
encoder, hparams, params = load_encoder_hparams_and_params(MODEL_SIZE, MODELS_DIR)
print("Model loaded successfully!")

Loading GPT-2 124M model...


Fetching checkpoint: 1.00kb [00:00, 2.98Mb/s]                                                       
Fetching encoder.json: 1.04Mb [00:00, 2.60Mb/s]                                                     
Fetching hparams.json: 1.00kb [00:00, 4.52Mb/s]                                                     
Fetching model.ckpt.data-00000-of-00001: 498Mb [00:30, 16.3Mb/s]                                    
Fetching model.ckpt.index: 6.00kb [00:00, 7.34Mb/s]                                                 
Fetching model.ckpt.meta: 472kb [00:00, 1.67Mb/s]                                                   
Fetching vocab.bpe: 457kb [00:00, 1.56Mb/s]                                                         


Model loaded successfully!


In [9]:
# Display model hyperparameters
print("Model Hyperparameters:")
print(f"  - Number of layers (n_layer): {hparams['n_layer']}")
print(f"  - Number of attention heads (n_head): {hparams['n_head']}")
print(f"  - Embedding dimension (n_embd): {hparams['n_embd']}")
print(f"  - Vocabulary size (n_vocab): {hparams['n_vocab']}")
print(f"  - Context length (n_ctx): {hparams['n_ctx']}")

Model Hyperparameters:
  - Number of layers (n_layer): 12
  - Number of attention heads (n_head): 12
  - Embedding dimension (n_embd): 768
  - Vocabulary size (n_vocab): 50257
  - Context length (n_ctx): 1024


## 7. 推論の実行

テキストを生成してみましょう！プロンプトと生成するトークン数を変更できます。

In [10]:
# Input prompt
prompt = "Alan Turing theorized that computers would one day become"

# Number of tokens to generate
n_tokens_to_generate = 40

print(f"Prompt: {prompt}")
print(f"Generating {n_tokens_to_generate} tokens...")
print()

Prompt: Alan Turing theorized that computers would one day become
Generating 40 tokens...



In [11]:
# Encode the input prompt
input_ids = encoder.encode(prompt)
print(f"Input token IDs: {input_ids}")
print(f"Number of input tokens: {len(input_ids)}")

# Make sure we don't exceed the context length
assert len(input_ids) + n_tokens_to_generate < hparams["n_ctx"], \
    f"Total tokens ({len(input_ids) + n_tokens_to_generate}) exceeds context length ({hparams['n_ctx']})"

Input token IDs: [36235, 39141, 18765, 1143, 326, 9061, 561, 530, 1110, 1716]
Number of input tokens: 10


In [12]:
# Generate output tokens
output_ids = generate(input_ids, params, hparams["n_head"], n_tokens_to_generate)

# Decode the generated tokens back to text
output_text = encoder.decode(output_ids)

print("\n" + "="*50)
print("Generated Text:")
print("="*50)
print(f"{prompt}{output_text}")

Generating: 100%|██████████| 40/40 [01:14<00:00,  1.85s/it]


Generated Text:
Alan Turing theorized that computers would one day become the most powerful machines on the planet.

The computer is a machine that can perform complex calculations, and it can perform these calculations in a way that is very similar to the human brain.



## 8. インタラクティブ生成

下で異なるプロンプトを試してみてください！

In [13]:
def generate_text(prompt, n_tokens=40):
    """Helper function to generate text from a prompt."""
    input_ids = encoder.encode(prompt)

    if len(input_ids) + n_tokens >= hparams["n_ctx"]:
        print(f"Warning: Reducing tokens to fit context length")
        n_tokens = hparams["n_ctx"] - len(input_ids) - 1

    output_ids = generate(input_ids, params, hparams["n_head"], n_tokens)
    output_text = encoder.decode(output_ids)

    return prompt + output_text

In [14]:
# Try your own prompts!
my_prompt = "The future of artificial intelligence is"
result = generate_text(my_prompt, n_tokens=50)
print(result)

Generating: 100%|██████████| 50/50 [01:29<00:00,  1.80s/it]

The future of artificial intelligence is uncertain.

"We're not sure what the future will look like," said Dr. Michael S. Schoenfeld, a professor of computer science at the University of California, Berkeley. "But we're not sure what the future will look


In [15]:
# Another example
my_prompt = "In a world where robots"
result = generate_text(my_prompt, n_tokens=50)
print(result)

Generating: 100%|██████████| 50/50 [01:27<00:00,  1.76s/it]

In a world where robots are becoming more and more commonplace, it's important to remember that robots are not just a threat to humanity, but also to the planet.

The robots that are currently in use are the ones that are currently being used to make the most of


## 9. モデルアーキテクチャの理解

モデルのパラメータを探索して、その構造をより深く理解しましょう。

In [16]:
# Explore the parameter structure
print("Top-level parameters:")
for key in params.keys():
    if key != 'blocks':
        if isinstance(params[key], dict):
            print(f"  {key}: {list(params[key].keys())}")
        else:
            print(f"  {key}: shape = {params[key].shape}")

print(f"\nNumber of transformer blocks: {len(params['blocks'])}")

Top-level parameters:
  ln_f: ['b', 'g']
  wpe: shape = (1024, 768)
  wte: shape = (50257, 768)

Number of transformer blocks: 12


In [18]:
# Explore a single transformer block
block = params['blocks'][0]
print("Structure of a transformer block:")
for key, value in block.items():
    if isinstance(value, dict):
        print(f"  {key}:")
        for k, v in value.items():
            # Check if v is also a dictionary, if so, iterate further
            if isinstance(v, dict):
                print(f"    {k}:")
                for sub_k, sub_v in v.items():
                    if isinstance(sub_v, np.ndarray):
                        print(f"      {sub_k}: shape = {sub_v.shape}")
                    else:
                        print(f"      {sub_k}: {type(sub_v)}")
            elif isinstance(v, np.ndarray): # if v is a numpy array directly
                print(f"    {k}: shape = {v.shape}")
            else: # Fallback for unexpected types for v
                print(f"    {k}: {type(v)}")
    elif isinstance(value, np.ndarray): # if value is a numpy array directly
        print(f"  {key}: shape = {value.shape}")
    else: # Fallback for unexpected types for value
        print(f"  {key}: {type(value)}")

Structure of a transformer block:
  attn:
    c_attn:
      b: shape = (2304,)
      w: shape = (768, 2304)
    c_proj:
      b: shape = (768,)
      w: shape = (768, 768)
  ln_1:
    b: shape = (768,)
    g: shape = (768,)
  ln_2:
    b: shape = (768,)
    g: shape = (768,)
  mlp:
    c_fc:
      b: shape = (3072,)
      w: shape = (768, 3072)
    c_proj:
      b: shape = (768,)
      w: shape = (3072, 768)


In [19]:
# Calculate total number of parameters
def count_params(d):
    total = 0
    for key, value in d.items():
        if isinstance(value, dict):
            total += count_params(value)
        elif isinstance(value, list):
            for item in value:
                if isinstance(item, dict):
                    total += count_params(item)
        elif isinstance(value, np.ndarray):
            total += value.size
    return total

total_params = count_params(params)
print(f"Total number of parameters: {total_params:,}")
print(f"Approximately: {total_params / 1e6:.1f}M parameters")

Total number of parameters: 124,439,808
Approximately: 124.4M parameters


## 10. トークナイザーの探索

BPEトークナイザーがどのように動作するか見てみましょう。

In [20]:
# Encode and decode examples
test_texts = [
    "Hello, world!",
    "GPT-2 is a large language model.",
    "The quick brown fox jumps over the lazy dog.",
]

for text in test_texts:
    tokens = encoder.encode(text)
    decoded = encoder.decode(tokens)
    print(f"Original: '{text}'")
    print(f"Tokens: {tokens}")
    print(f"Decoded: '{decoded}'")
    print(f"Number of tokens: {len(tokens)}")
    print()

Original: 'Hello, world!'
Tokens: [15496, 11, 995, 0]
Decoded: 'Hello, world!'
Number of tokens: 4

Original: 'GPT-2 is a large language model.'
Tokens: [38, 11571, 12, 17, 318, 257, 1588, 3303, 2746, 13]
Decoded: 'GPT-2 is a large language model.'
Number of tokens: 10

Original: 'The quick brown fox jumps over the lazy dog.'
Tokens: [464, 2068, 7586, 21831, 18045, 625, 262, 16931, 3290, 13]
Decoded: 'The quick brown fox jumps over the lazy dog.'
Number of tokens: 10

